<a href="https://colab.research.google.com/github/friedmam/work_availability_scheduler_2025/blob/main/Honors_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup (for editing)

If you have made any changes to the Excel file, please make sure to replicate them here! When you're done, hide this tab and the next, so that the first code block that shows up is under "Run command to upload files". Press the small "run" button (the circle with the triangle inside; if you aren't hovering over it, it will look like this: "[ ]") on the hidden cells, and then run the visible commands in order!

Enjoy!

In [1]:
# How to read the personal-info block on each Excel sheet
# ──────────────────────────────────────────────────────────────
# Each field is mapped to the (row, col) pair. If a cell is merged, reference the top left corner.

PERSONAL_FIELDS = dict(
    name            = (4, 2),
    max_hours_week  = (4, 7),
    max_hours_day   = (6, 7),
    is_td           = (5, 11),
    is_hd           = (8, 11),
    is_manager      = (11,11),
    class_year      = (19,10),
)

# Where to find the big availability grids inside a sheet
# ──────────────────────────────────────────────────────────────
# Change these if the template ever moves around
# Items are inclusive, then exclusive;
#   so the last number should be one after the actual end of the section


# Location of schedule for Help Desk
HD_GRID  = dict(rows=list(range(32,39)),
                cols=list(range(3,8)),
                day_row=11,
                time_col=1)

# Location of schedule for Tech Desk and Supply Runner
REG_GRID = dict(
    day_cols = list(range(2,9)),
    day_row = 11,
    time_col = 1,
    row_ranges = [                       # one sub-list per weekday (Su … Sa)
        list(range(17,28)),              # Sunday
        list(range(13,28)),              # Monday
        list(range(13,28)),              # Tuesday
        list(range(13,28)),              # Wednesday
        list(range(13,28)),              # Thursday
        list(range(13,24)),              # Friday
        list(range(17,24)),              # Saturday
    ],
)

# Pre-defined shift break-points for each role
# ──────────────────────────────────────────────────────────────
TECH_DESK_STOPS = {
    'Sunday':    ['12:00','13:00','14:30','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00'],
    'Monday':    ['10:00','11:00','12:00','13:00','14:30','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00'],
    'Tuesday':   ['10:00','11:00','12:00','13:00','14:30','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00'],
    'Wednesday': ['10:00','11:00','12:00','13:00','14:30','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00'],
    'Thursday':  ['10:00','11:00','12:00','13:00','14:30','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00'],
    'Friday':    ['10:00','11:00','12:00','13:00','14:30','16:00','17:00','18:00'],
    'Saturday':  ['12:00','13:00','14:30','16:00','17:00','18:00'],
}

SUPPLY_RUN_STOPS = {
    'Sunday': [
        ('14:30', '16:00'), # start time, end time
        ('18:00', '19:00'), # start time, end time
        ('20:00', '21:00'), # start time, end time
        ('22:00', '23:00'), # start time, end time
    ],
    'Monday': [
        ('09:00', '10:00'),
        ('11:00', '12:00'),
        ('14:30', '16:00'),
        ('18:00', '19:00'),
        ('20:00', '21:00'),
        ('22:00', '23:00'),
    ],
    'Tuesday': [
        ('09:00', '10:00'),
        ('11:00', '12:00'),
        ('14:30', '16:00'),
        ('18:00', '19:00'),
        ('20:00', '21:00'),
        ('22:00', '23:00'),
    ],
    'Wednesday': [
        ('09:00', '10:00'),
        ('11:00', '12:00'),
        ('14:30', '16:00'),
        ('18:00', '19:00'),
        ('20:00', '21:00'),
        ('22:00', '23:00'),
    ],
    'Thursday': [
        ('09:00', '10:00'),
        ('11:00', '12:00'),
        ('14:30', '16:00'),
        ('18:00', '19:00'),
        ('20:00', '21:00'),
        ('22:00', '23:00'),
    ],
    'Friday': [
        ('09:00', '10:00'),
        ('11:00', '12:00'),
        ('14:30', '16:00'),
    ],
    'Saturday': [
        ('14:30', '16:00'),
    ],
}

DAY_ORDER = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

# Setup (not for editing)

## Imports

In [2]:
import os, glob, io, re, sys, zipfile, random, math
from datetime import datetime, time as dtime, timedelta
from collections import defaultdict

import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.exceptions import InvalidFileException

import shutil

# Colab-specific helpers
from google.colab import files
import ipywidgets as w
from IPython.display import display, clear_output

## Classes

In [3]:
class StuWorker:
    """One student worker and all their personal data / availability."""
    def __init__(self, name, max_hours_week, max_hours_day,
                 availability, hd_availability,
                 class_year=1, is_manager=False,
                 is_hd=False, is_td=False, is_sr=True):
        self.name = name
        self.max_hours_week = max_hours_week
        self.max_hours_day = max_hours_day
        self.availability = availability or []
        self.hd_availability = hd_availability or []
        self.is_manager = is_manager
        self.class_year = class_year
        self.is_hd = is_hd
        self.is_td = is_td
        self.is_sr = is_sr

class TimeBlock:
    """A single contiguous chunk of time (e.g. 10:00–11:00 on Monday)."""
    def __init__(self, start, end, day, available=3):
        self.start_time = parse_hhmm(start) if isinstance(start, str) else start
        self.end_time = parse_hhmm(end)   if isinstance(end,   str) else end
        self.day = day
        self.available = available        # 1 = preferred, 2 = okay, 3 = unavailable

class ScheduleTimeBlock:
    """Same as TimeBlock, plus fields the scheduler fills in later."""
    def __init__(self, start, end, day, stu_available=0, current_color='#ffffff', sid=""):
        self.start_time = parse_hhmm(start) if isinstance(start, str) else start
        self.end_time = parse_hhmm(end)   if isinstance(end,   str) else end
        self.day = day
        self.stu_available = stu_available
        self.current_color = current_color
        self.id = sid           # which student took it (blank ⇒ open)

## Utility Functions

In [4]:
def timeblock_to_dict(tb):
  return {
    "start_time": tb.start_time,
    "end_time": tb.end_time,
    "day": tb.day,
    "available": tb.available,
  }

def dict_to_timeblock(d):
  return TimeBlock(
    start_time=d["start_time"],
    end_time=d["end_time"],
    day=d["day"],
    available=d["available"],
  )

def scheduletimeblock_to_dict(tb):
  return {
    "start_time": tb.start_time,
    "end_time": tb.end_time,
    "day": tb.day,
    "stu_available": tb.stu_available,
    "current_color": tb.current_color,
    "id": tb.id,
  }

def dict_to_scheduletimeblock(d):
  return ScheduleTimeBlock(
    start_time=d["start_time"],
    end_time=d["end_time"],
    day=d["day"],
    stu_available=d.get("stu_available", 0),
    current_color=d.get("current_color", '#ffffff'),
    id=d["id"]
  )

def stuworker_to_dict(stu):
  return {
    "name": stu.name,
    "max_hours_week": stu.max_hours_week,
    "max_hours_day": stu.max_hours_day,
    "availability": [timeblock_to_dict(tb) for tb in stu.availability],
    "hd_availability": [timeblock_to_dict(tb) for tb in stu.hd_availability],
    "class_year": stu.class_year,
    "is_manager": stu.is_manager,
    "is_hd": stu.is_hd,
    "is_td": stu.is_td,
    "is_sr": stu.is_sr
  }

def dict_to_stuworker(d):
  availability = [dict_to_timeblock(tb) for tb in d.get("availability", [])]
  hd_availability = [dict_to_timeblock(tb) for tb in d.get("hd_availability", [])]
  return StuWorker(
    name=d["name"],
    max_hours_week=d["max_hours_week"],
    max_hours_day=d["max_hours_day"],
    availability=availability,
    hd_availability=hd_availability,
    class_year=d["class_year"],
    is_manager=d["is_manager"],
    is_hd=d["is_hd"],
    is_td=d["is_td"],
    is_sr=d["is_sr"]
  )

def parse_time_str(t_str):
  return datetime.strptime(t_str.strip(), "%H:%M").time()

def parse_hhmm(t):
  if isinstance(t, dtime):
    return t
  return datetime.strptime(t, "%H:%M").time()

def times_overlap(start1, end1, start2, end2):
  # Overlap if start1 < end2 and start2 < end1
  return (start1 < end2) and (start2 < end1)

def student_already_assigned_in_other_schedules(stu, day, start_str, end_str, role_flag):
  '''
  Check if the student is assigned to a shift in a different schedule.
  '''
  # Exclude a schedule that matches role_flag
  schedules_to_check = []
  if role_flag != 'is_hd':
    schedules_to_check.append(HD_SCHEDULE)
  if role_flag != 'is_td':
    schedules_to_check.append(TD_SCHEDULE)
  if role_flag != 'is_sr':
    schedules_to_check.append(SR_SCHEDULE)

  this_start = parse_time_str(start_str)
  this_end   = parse_time_str(end_str)

  for schedule_list in schedules_to_check:
    for sched_block in schedule_list:
      if sched_block.id == stu.name and sched_block.day.strip() == day.strip():
        other_start = parse_time_str(sched_block.start_time)
        other_end   = parse_time_str(sched_block.end_time)
        if times_overlap(this_start, this_end, other_start, other_end):
          return True

  return False

def _field(sheet, key):
  r, c = PERSONAL_FIELDS[key]
  return sheet.cell(row=r, column=c).value

## Read Excel Files

In [5]:
def safe_time_cell(cell):
    if cell in (None, ""):
        return None
    if isinstance(cell, (datetime, dtime)):
        return cell.strftime("%H:%M")
    return str(cell)

def get_time_blocks(sheet, row_ranges, col_range, day_row, time_col, is_hd=False):
  time_blocks = []

  def avail_val(v):
    # Accept '1' or '2'; everything else -> 3 (unavailable)
    if v is None:
      return 3
    try:
      v_int = int(str(v).strip())   # " 1 " ➜ 1
    except ValueError:
      return 3
    return v_int if v_int in (1, 2) else 3

  if is_hd:
    rows = row_ranges[0]                       # HD rows list
    for c in col_range:
      day = sheet.cell(row=day_row, column=c).value
      for i in range(len(rows) - 1):
        # raw cells
        start_raw = sheet.cell(row=rows[i],   column=time_col).value
        end_raw   = sheet.cell(row=rows[i+1], column=time_col).value

        # convert – None/"" becomes None
        start_time = safe_time_cell(start_raw)
        end_time   = safe_time_cell(end_raw)

        # skip any incomplete pairs
        if not start_time or not end_time:
          continue

        time_blocks.append(
          TimeBlock(start_time, end_time, day,
                    available=avail_val(sheet.cell(row=rows[i], column=c).value))
        )

  else:
    # Regular availability
    for rows_for_day, c in zip(row_ranges, col_range):
      day_name = sheet.cell(row=day_row, column=c).value.strip()
      for i in range(len(rows_for_day) - 1):
        start_row, end_row = rows_for_day[i], rows_for_day[i+1]
        start_raw = sheet.cell(row=start_row, column=time_col).value
        end_raw   = sheet.cell(row=end_row,   column=time_col).value

        start_time = safe_time_cell(start_raw)
        end_time   = safe_time_cell(end_raw)

        if not start_time or not end_time:
          continue

        time_blocks.append(
          TimeBlock(start_time, end_time, day_name,
                    available=avail_val(sheet.cell(row=start_row, column=c).value))
        )

  return time_blocks

def _open_workbook_safe(path):
  """
  Try to open the Excel sheet and return it or return None.
  """
  try:
    wb = load_workbook(path, data_only=True)
  except InvalidFileException as e:
    print(f"❌  {os.path.basename(path)} is not a valid .xlsx: {e}")
    return None
  except Exception as e:
    print(f"❌  Could not open {os.path.basename(path)}: {e}")
    return None

  if not wb.sheetnames:
    # sometimes mildly corrupted files load only in read‑only mode
    try:
      wb = load_workbook(path, read_only=True, data_only=True)
    except Exception:
      pass

  if not wb.sheetnames:
    print(f"⚠️  {os.path.basename(path)} contains 0 visible worksheets."
          "  Open it in Excel/LibreOffice, verify at least one sheet is"
          " visible, then save again as .xlsx.")
    return None

  return wb.active

def make_stuworker(file_path):
  """
  Reads one .xlsx file and returns the constructed StuWorker instance,
  or None on failure.  Uses PERSONAL_FIELDS / HD_GRID / REG_GRID
  """
  sheet = _open_workbook_safe(file_path)
  if sheet is None:
    return None

  # Field Look-Ups
  name = _field(sheet, "name")
  max_hours_week = _field(sheet, "max_hours_week")
  max_hours_day = _field(sheet, "max_hours_day")
  is_td = _field(sheet, "is_td")
  is_hd = _field(sheet, "is_hd")
  is_manager = _field(sheet, "is_manager")
  class_year = _field(sheet, "class_year")

  # Availability Grids
  hd_availability = get_time_blocks(
    sheet,
    [HD_GRID["rows"]],
    HD_GRID["cols"],
    day_row  = HD_GRID["day_row"],
    time_col = HD_GRID["time_col"],
    is_hd    = True
  )

  availability = get_time_blocks(
    sheet,
    REG_GRID["row_ranges"],
    REG_GRID["day_cols"],
    day_row  = REG_GRID["day_row"],
    time_col = REG_GRID["time_col"]
  )

  # Make the StuWorker object
  stu = StuWorker(
    name, max_hours_week, max_hours_day,
    availability, hd_availability,
    class_year, is_manager, is_hd, is_td
  )

  # print(f"✅  Added {stu.name}  ←  {os.path.basename(file_path)}")
  # print("HD blocks:", len(hd_availability), "Regular blocks:", len(availability))
  return stu

def scrape_xlsx_files(file_paths):
  """
  Parse each Excel file and build StuWorker objects.
  """
  return [stu for p in file_paths
        if p.endswith(".xlsx")
        for stu in (make_stuworker(p),)
        if stu] # keep only not‑None values

## Upload Excel files from computer

In [6]:
def upload_excel_files():
  uploaded = files.upload() # user (Robert) chooses files
  written  = []

  for name, data in uploaded.items():
    # normalise base name: strip any " (n)"
    base, ext = os.path.splitext(name)
    base = re.sub(r" \(\d+\)$", "", base) # remove  " (3)"  etc. for duplicate removal
    clean_name = f"{base}{ext}"

    # remove *all* existing variants
    for old in glob.glob(f"{base}*.xlsx"):
      os.remove(old)

    # write fresh file with the correct name
    with open(clean_name, "wb") as f:
      f.write(data)
    written.append(clean_name)
  return written


## Edit Student Roles and Information

In [7]:
# helpers
def missing_fields(stu):
  miss = []
  if not stu.name: miss.append("name")
  if stu.class_year not in (1, 2, 3, 4): miss.append("class_year (1‑4)")
  if not stu.max_hours_week: miss.append("max_hours_week")
  if not stu.max_hours_day: miss.append("max_hours_day")
  for flag in ("is_manager", "is_hd", "is_td", "is_sr"):
    if getattr(stu, flag) not in (True, False):
      miss.append(flag)
  return miss

def tiny(label=" "):        # 1‑char label keeps checkboxes visible in Colab. If you delete this, the check boxes will stop showing up.
  return dict(description=label, indent=False)

warn_out = w.Output()

def refresh_warning():
  with warn_out:
    clear_output(wait=True)
    bad = {s.name: missing_fields(s) for s in ALL_STUDENTS if missing_fields(s)}
    if not bad:
      display(w.HTML("<b style='color:green'>All records complete.</b>"))
    else:
      items = "".join(f"<li><b>{n}</b>: {', '.join(f)}</li>" for n, f in bad.items())
      display(w.HTML(f"<b style='color:red'>⚠ Missing info:</b><ul>{items}</ul>"))

# Build table
student_list_box = w.VBox()

def build_student_row(stu, idx):
  """Return an HBox with editing widgets + up/down buttons for this student."""
  # order buttons
  btn_up   = w.Button(description="▲",   layout=w.Layout(width='28px'))
  btn_down = w.Button(description="▼", layout=w.Layout(width='28px'))

  def move(delta):
    new_pos = idx + delta
    if 0 <= new_pos < len(ALL_STUDENTS):
      ALL_STUDENTS[idx], ALL_STUDENTS[new_pos] = ALL_STUDENTS[new_pos], ALL_STUDENTS[idx]
      refresh_student_table()         # redraw with new ordering

  btn_up.on_click(lambda *_: move(-1))
  btn_down.on_click(lambda *_: move(+1))

  # editable fields
  name = w.Text(value=stu.name or "", layout=w.Layout(width='120px'))
  year = w.Dropdown(value=stu.class_year or 1,  options=[1,2,3,4], layout=w.Layout(width='55px'))
  max_wk = w.BoundedIntText(value=stu.max_hours_week or 0, min=0, max=15, layout=w.Layout(width='70px'))
  max_day = w.BoundedIntText(value=stu.max_hours_day or 0, min=0, max=15, layout=w.Layout(width='70px'))
  mgr = w.Checkbox(value=bool(stu.is_manager), **tiny(), layout=w.Layout(width='36px'))
  hd  = w.Checkbox(value=bool(stu.is_hd), **tiny(), layout=w.Layout(width='36px'))
  td  = w.Checkbox(value=bool(stu.is_td), **tiny(), layout=w.Layout(width='36px'))
  sr  = w.Checkbox(value=bool(stu.is_sr), **tiny(), layout=w.Layout(width='36px'))

  #  live‑bind back to the StuWorker object  ────────────────
  def bind(widget, attr):
    widget.observe(lambda ch, a=attr: setattr(stu, a, ch["new"]), "value")
  for wdg, att in [(name,"name"), (year,"class_year"), (max_wk,"max_hours_week"),
                    (max_day,"max_hours_day"), (mgr,"is_manager"), (hd,"is_hd"),
                    (td,"is_td"), (sr,"is_sr")]:
    bind(wdg, att)
  # refresh warnings on any change
  for wdg in (name, year, max_wk, max_day, mgr, hd, td, sr):
    wdg.observe(lambda *_: refresh_warning(), "value")

  # highlight rows with missing data
  row = w.HBox([btn_up, btn_down, name, year, max_wk, max_day,
                w.HTML(" ", layout=w.Layout(width='8px')),  # spacer
                mgr, hd, td, sr])
  return row

def refresh_student_table():
  header = w.HBox([
      w.HTML("<b> </b>", layout=w.Layout(width='56px')),   # space for arrows
      w.HTML("<b>Name</b>",    layout=w.Layout(width='120px')),
      w.HTML("<b>Year</b>",    layout=w.Layout(width='55px')),
      w.HTML("<b>Max/wk</b>",  layout=w.Layout(width='70px')),
      w.HTML("<b>Max/day</b>", layout=w.Layout(width='70px')),
      w.HTML("<b> </b>",       layout=w.Layout(width='8px')),
      w.HTML("<b>Mgr</b>", layout=w.Layout(width='36px')),
      w.HTML("<b>HD</b>",  layout=w.Layout(width='36px')),
      w.HTML("<b>TD</b>",  layout=w.Layout(width='36px')),
      w.HTML("<b>SR</b>",  layout=w.Layout(width='36px')),
  ])
  rows = [header] + [build_student_row(s, i) for i, s in enumerate(ALL_STUDENTS)]
  student_list_box.children = rows

# ───────── optional automatic sorting ───────────────────────
def _available_blocks(stu):
  """Count blocks with Availability 1 or 2 in BOTH regular + HD lists."""
  cnt = 0
  for lst in (getattr(stu, "availability", []),
              getattr(stu, "hd_availability", [])):
    cnt += sum(tb.available in (1, 2) for tb in lst)
  return cnt

def _sort_key(stu):
  mgr_rank   = 0 if stu.is_manager else 1          # managers first
  class_rank = -stu.class_year                     # 4→‑4 (first), 1→‑1 (last)

  avail = _available_blocks(stu)
  hrs   = stu.max_hours_week or math.inf           # treat missing as infinite
  ratio = avail / hrs if hrs else float('inf')     # high ratio = good

  # Python sorts ascending
  return (
    mgr_rank,                # managers (0) before non‑managers (1)
    class_rank,              # seniors (-4) before juniors (-3)…
    -ratio,                  # larger ratio first  (negated for ascending sort)
    hrs,                     # fewer requested hours next
    -avail                   # finally, more raw blocks first
  )

btn_auto_sort = w.Button(description="Auto Sort (Mgr → Yr → Ratio)", icon='sort', layout=w.Layout(width='250px'))

def auto_sort(_=None):
    ALL_STUDENTS.sort(key=_sort_key)
    refresh_student_table()

btn_auto_sort.on_click(auto_sort)

## Schedule Visualization

In [8]:
def parse_hhmm(t):
  if isinstance(t, dtime):
    return t
  return datetime.strptime(t, "%H:%M").time()

def span_hours(start, end):
  """Return fractional hours in the half‑open interval [start,end)."""
  dt0, dt1 = parse_hhmm(start), parse_hhmm(end)
  delta = datetime.combine(datetime.today(), dt1) - datetime.combine(datetime.today(), dt0)
  return delta.total_seconds() / 3600.0

def times_overlap(a0, a1, b0, b1):
  return parse_hhmm(a0) < parse_hhmm(b1) and parse_hhmm(b0) < parse_hhmm(a1)

DAY_ORDER = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

def unique_hd_blocks(students):
  seen, out = set(), []
  for stu in students:
    for tb in (stu.hd_availability or []):
      key = (tb.day, tb.start_time, tb.end_time)
      if key not in seen:
        seen.add(key)
        out.append(ScheduleTimeBlock(tb.start_time, tb.end_time, tb.day))
  return out

def build_td_schedule(range_dict):
  '''Return a list[ScheduleTimeBlock] populated from TD_RANGE'''
  sched = []
  for day, stops in range_dict.items():
    for i in range(len(stops) - 1):
      start, end = stops[i], stops[i + 1]
      sched.append(
        ScheduleTimeBlock(start,end,day,0,"#ffffff","")
      )
  return sched

def build_sr_schedule(range_dict):
    """
    Convert SUPPLY_RUN_STOPS (dict[day] -> list[(start,end)]) into a list
    of ScheduleTimeBlock objects.
    """
    sched = []
    for day, blocks in range_dict.items():
        for start, end in blocks:
            sched.append(ScheduleTimeBlock(start, end, day))
            #print("timeblock start: ", start, ", end:", end, ", day: ", day)
    return sched

def build_hd_schedule(students):
    """Return Help‑Desk time blocks derived from students"""
    return unique_hd_blocks(students)

def matching_td_shift(day, start_time, td_sched):
  """
  Find the Tech Desk shift that starts at `start_time` on `day`.
  Returns the matching ScheduleTimeBlock, or None if not found.
  """
  return next(
    (td for td in td_sched if td.day == day and td.start_time == start_time),
    None
  )

# Scheduler State object & reliant helpers

class SchedulerState:
  """Container for the notebook‑wide scheduling state."""

  def __init__(self, students=None, hd_sched=None, td_sched=None, sr_sched=None, pass_style="ff"):
    # Roster may be missing at first → default to empty list
    self._students = students or []
    self.pass_style = pass_style          # "ff" or "fb"
    self.direction = +1                  # +1 → forward, −1 → backward
    self.round = 1
    self.current_index = {"hd": 0, "sr": 0, "td": 0}
    self.round_assigned = defaultdict(int)

    # Schedules dictionary may also start empty
    self.schedules = {
      "hd": hd_sched or [],
      "td": td_sched or [],
      "sr": sr_sched or [],
    }

  # ── dynamic roster handling ────────────────────────────────────────────
  def attach_students(self, students):
    """Inject a full roster *after* construction (also resets index)."""
    self._students = students
    for k in self.current_index:
      self.current_index[k] = min(self.current_index[k], len(students) - 1)

  # convenience alias – writable so legacy code can mutate list directly
  @property
  def ordered_students(self):
    return self._students

  @ordered_students.setter
  def ordered_students(self, new_list):
    self._students = new_list or []
    for k in self.current_index:
      self.current_index[k] = min(self.current_index[k], len(self._students) - 1 if self._students else 0)

  # ── helper used all over notebook ───────────────────────────────────────
  @property
  def cur_stu(self):
    if not self._students:
      return None
    role = role_toggle.value
    idx = self.current_index.get(role, 0)
    return self._students[idx] if idx < len(self._students) else None

  def worked_hours(self, stu):
    h = 0.0
    for sched in self.schedules.values():
      for tb in sched:
        if tb.id == stu.name:
          h += span_hours(tb.start_time, tb.end_time)
    return h

  def worked_hours_day(self, stu, day):
    h = 0.0
    for sched in self.schedules.values():
      for tb in sched:
        if tb.id == stu.name and tb.day == day:
          h += span_hours(tb.start_time, tb.end_time)
    return h

STATE = SchedulerState()

def _has_open_block(state: STATE, stu, role_key: str) -> bool:
  """
  Checks that at least one currently empty block exists in state.schedules[role_key]
  that the student could legally take right now (making sure availability,
  no overlaps, and hour limits are okay).
  """
  wk = state.worked_hours(stu)

  for tb in state.schedules[role_key]:
    if tb.id: # already filled
      continue

    # would this block push the student over weekly / daily limits?
    dur = span_hours(tb.start_time, tb.end_time)
    if wk + dur > stu.max_hours_week:
      continue
    if state.worked_hours_day(stu, tb.day) + dur > stu.max_hours_day:
      continue

    # overlap with something the student is already working?
    clash = any(
      times_overlap(tb.start_time, tb.end_time, other.start_time, other.end_time)
      for sched in state.schedules.values() # every role
      for other in sched
      if other.id == stu.name and other.day == tb.day
    )
    if clash:
      continue

    # availability check (HD vs TD vs SR)
    if role_key == "hd":
      source = stu.hd_availability
      ok = any(sb.day == tb.day and sb.start_time == tb.start_time and
               sb.end_time == tb.end_time and sb.available in (1, 2)
               for sb in source)

    elif role_key == "td":
      source = stu.availability
      ok = any(sb.day == tb.day and sb.start_time == tb.start_time and
               sb.end_time == tb.end_time and sb.available in (1, 2)
               for sb in source)

    else:  # supply‑runner needs the matching TD block first
      td_match = matching_td_shift(tb.day, tb.start_time, state.schedules["td"])
      ok = False
      if td_match:
        ok = any(av.day == td_match.day and
                 av.start_time == td_match.start_time and
                 av.end_time == td_match.end_time and
                 av.available in (1, 2)
                 for av in stu.availability)
      # 09‑10 fallback
      if not ok and tb.start_time == dtime(9, 0) and tb.end_time == dtime(10, 0):
        ok = any(av.day == tb.day and av.start_time == tb.start_time and
                 av.end_time == tb.end_time and av.available in (1, 2)
                 for av in stu.availability)

    if ok:
      return True                        # found at least one valid slot

  return False

def visible_students(state, role_key):
  #Return STATE.ordered_students filtered appropriately
  flag = f"is_{role_key}"           # -> 'is_hd' | 'is_sr' | 'is_td'
  roster = []

  # always start from the full underlying list, *not* a previously‑filtered copy
  for stu in state._students:
    if not getattr(stu, flag):
      continue
    if state.worked_hours(stu) >= stu.max_hours_week:
      continue
    if not _has_open_block(state, stu, role_key):
      continue
    roster.append(stu)

  return roster

def initialise_scheduler(roster, td_stops, sr_stops, pass_style="ff"):
  """Return a ready‑to‑use SchedulerState once *roster* is known."""
  hd_sched = build_hd_schedule(roster)
  td_sched = build_td_schedule(td_stops)
  sr_sched = build_sr_schedule(sr_stops)

  state = SchedulerState(
    students=[s for s in roster if s.is_sr],  # start with SR filter
    hd_sched=hd_sched,
    td_sched=td_sched,
    sr_sched=sr_sched,
    pass_style=pass_style,
  )
  return state

#  Widgets — list of students, schedule grid, control panel

#  List of students
def build_student_list(role_key):
  flag = f"is_{role_key}"
  items = []
  visible_students = [s for s in STATE.ordered_students if getattr(s, flag)]

  for idx, stu in enumerate(visible_students):
    weekly = STATE.worked_hours(stu)
    daily  = max(STATE.worked_hours_day(stu, d) for d in DAY_ORDER)
    color  = "red" if weekly >= stu.max_hours_week else \
              "#ffd166" if stu is STATE.cur_stu else "black"
    items.append(
        w.HTML(f"<span style='color:{color}'>{stu.name}"
                f"&nbsp;({weekly:.1f}/{stu.max_hours_week} wk, "
                f"{daily:.1f}/{stu.max_hours_day} day)</span>")
    )
  return w.VBox(items, layout=w.Layout(border='1px solid gray', width='280px', overflow='auto'))

def is_on_hour_or_half(t_):
  """
  Accepts either a 'HH:MM' string or a datetime.time object.
  Returns True if the minutes are exactly 00 or 30.
  """
  if isinstance(t_, str): # convert "14:30" ➜ datetime.time
    t_ = datetime.strptime(t_.strip(), "%H:%M").time()

  return t_.minute in (0, 30)

def unassign_student_from_role(stu, role_key):
  """
  Remove 'stu' from every time-block in STATE.schedules[role_key] and
  roll back the per-round counter.
  """
  removed = 0
  for tb in STATE.schedules[role_key]:
    if tb.id == stu.name:
      tb.id = ""
      removed += 1

  if removed:
    STATE.round_assigned[stu.name] = max(0, STATE.round_assigned.get(stu.name, 0) - removed)

# Schedule grid for a role
def build_grid(role_key):
  sched = STATE.schedules[role_key]
  if not sched:
    return w.HTML("No time‑blocks defined for this role.")
  # extract unique times & days in deterministic order
  days  = sorted({tb.day for tb in sched}, key=DAY_ORDER.index)
  times = sorted({tb.start_time for tb in sched}, key=parse_hhmm)

  from ipywidgets import GridspecLayout, Button, Layout
  grid = GridspecLayout(len(times)+1, len(days)+1,
                        grid_gap="2px", layout=Layout(width='100%'))
  # headers
  grid[0,0] = w.HTML("<b>Time</b>")
  for c,day in enumerate(days,1):
    grid[0,c] = w.HTML(f"<b>{day}</b>")
  for r,t in enumerate(times,1):
    grid[r,0] = w.HTML(f"<b>{t}</b>")

  # mapping for quick lookup
  block_lookup = {(tb.day,tb.start_time):tb for tb in sched}

  # cell builder
  def cell_content(tb):
    if tb.id:
        return tb.id
    # else show availability count
    avail = 0
    for stu in STATE.ordered_students:
      # skip if already scheduled elsewhere overlapping
      clash=False
      for sc in STATE.schedules.values():
        for tb2 in sc:
          if tb2.id==stu.name and tb2.day==tb.day and times_overlap(tb.start_time,tb.end_time,tb2.start_time,tb2.end_time):
            clash=True; break
        if clash: break
      if clash: continue
      # check if stu lists this block as available
      source = stu.hd_availability   if role_key=="hd" else stu.availability

      if role_key == "sr":
        matching_td_block = matching_td_shift(tb.day, tb.start_time, STATE.schedules["td"])
        matched = False
        if matching_td_block:
          for av in stu.availability:
            if (av.day == matching_td_block.day and
              av.start_time == matching_td_block.start_time and
              av.end_time == matching_td_block.end_time and
              av.available in (1, 2)):
              avail += 1
              matched = True
              break
        if not matched:
          # Check direct match with this SR block
          for av in stu.availability:
            if (av.day == tb.day and
              av.start_time == dtime(9,0) and
              av.end_time == dtime(10,0) and
              av.available in (1, 2)):
              avail += 1
              break
      else:
        for sb in source:
          if sb.day == tb.day and sb.start_time == tb.start_time and sb.end_time == tb.end_time and sb.available in (1, 2):
            avail += 1
            break
    return str(avail)

  # click handler
  def make_handler(tb, role_key):
    def _on_click(b):
      if tb.id:  # If it's already filled → unassign the person from the block
        stu_name = tb.id
        tb.id = ""
        STATE.round_assigned[stu_name] = max(0, STATE.round_assigned.get(stu_name, 0) - 1)
        refresh_ui()
        return

      stu = STATE.cur_stu
      if stu is None:
        return

      # weekly / daily limits
      if STATE.worked_hours(stu) + span_hours(tb.start_time, tb.end_time) > stu.max_hours_week:
        return
      if STATE.worked_hours_day(stu, tb.day) + span_hours(tb.start_time, tb.end_time) > stu.max_hours_day:
        return

      # overlap test
      for sc in STATE.schedules.values():
        for other in sc:
          if (other.id == stu.name and other.day == tb.day and
              times_overlap(tb.start_time, tb.end_time, other.start_time, other.end_time)):
            return

      # ok. assign
      tb.id = stu.name
      STATE.round_assigned[stu.name] += 1
      refresh_ui()
    return _on_click


  # fill cells
  for r, t in enumerate(times, 1):
    for c, day in enumerate(days, 1):
      tb = block_lookup.get((day, t))
      if tb is None:
        # no shift is defined for that day/time → show an empty, disabled button
        grid[r, c] = w.Button(description='', disabled=True, layout=Layout(width='80px', height='34px'))
        continue

      style = '' # default (grey)
      cur = STATE.cur_stu
      if cur and not tb.id: # show only on vacant blocks
          source = (cur.hd_availability if role_key == 'hd' else cur.availability)
          avail_lvl = None
          if role_key == "sr":
            # Find the matching Tech Desk shift
            matching_td_block = matching_td_shift(tb.day, tb.start_time, STATE.schedules["td"])
            if matching_td_block:
              for sb in source:
                if (sb.day == matching_td_block.day and
                  sb.start_time == matching_td_block.start_time and
                  sb.end_time == matching_td_block.end_time):
                  avail_lvl = sb.available
                  break
          else:
            for sb in source:
              if (sb.day == day and sb.start_time == t and sb.end_time == tb.end_time):
                avail_lvl = sb.available
                break
          if avail_lvl == 1:
            style = 'success' # green
          elif avail_lvl == 2:
            style = 'warning' # yellow

      b = w.Button(description=cell_content(tb), button_style=style, layout=Layout(width='80px', height='34px'))
      b.on_click(make_handler(tb, role_key))
      grid[r, c] = b
  return grid

# round counter & navigation buttons
round_label  = w.HTML()
def btn_next(_):
  _step(+1)
def btn_prev(_):
  _step(-1)
next_btn = w.Button(description="Next ▶")
prev_btn = w.Button(description="◀ Prev")
next_btn.on_click(btn_next); prev_btn.on_click(btn_prev)

auto_btn = w.Button(description="Automate", button_style="info")
auto_btn.on_click(lambda _: auto_assign_student(True))

def _step(delta):
  roster = visible_students(STATE, role_toggle.value)
  if not roster:
    return

  # Where are we *within the filtered list*?
  try:
    pos = roster.index(STATE.cur_stu)
  except ValueError:
    pos = -1 if delta > 0 else 0

  pos = (pos + delta) % len(roster)

  STATE.current_index[role_toggle.value] = STATE.ordered_students.index(roster[pos])
  refresh_ui()

# Automation of shift assignments for an individual student
def auto_assign_student(repeat):
  stu = STATE.cur_stu
  role_key = role_toggle.value
  if stu is None:
    return

  def all_blocks():
    return STATE.schedules[role_key]

  # Helper: filter eligible blocks
  def eligible_blocks():
    blocks = []
    for tb in all_blocks():
      if tb.id: continue  # Already filled

      source = stu.hd_availability if role_key == "hd" else stu.availability
      if role_key == "sr":
          # ↳ SR shifts are mapped to the *corresponding* TD shift first
          td = matching_td_shift(tb.day, tb.start_time)
          def is_available(block, start, end):
              return (block.day == start.day and
                      block.start_time == start.start_time and
                      block.end_time   == start.end_time and
                      block.available in (1,2))
          avail_match = None
          if td:
              avail_match = next((x for x in stu.availability
                                   if is_available(x, td, td)), None)
          if not avail_match:
              # fall back to exact SR match (handles 09:00–10:00 block)
              avail_match = next((x for x in stu.availability
                                   if is_available(x, tb, tb)), None)
      else:
          avail_match = next((x for x in source
                               if x.day == tb.day
                               and x.start_time == tb.start_time
                               and x.end_time   == tb.end_time
                               and x.available in (1,2)), None)
      if not avail_match or avail_match.available == 3:
        continue  # Unavailable

      # Skip if already scheduled during this block
      for sc in STATE.schedules.values():
        for other in sc:
          if other.id == stu.name and other.day == tb.day and times_overlap(tb.start_time, tb.end_time, other.start_time, other.end_time):
            break
        else:
          continue
        break
      else:
        # Weekly/daily limits
        if STATE.worked_hours(stu) + span_hours(tb.start_time, tb.end_time) > stu.max_hours_week:
          continue
        if STATE.worked_hours_day(stu, tb.day) + span_hours(tb.start_time, tb.end_time) > stu.max_hours_day:
          continue

        # 10‑hour sleep window enforcement
        all_times = []
        for sched in STATE.schedules.values():
          for b in sched:
            if b.id == stu.name:
              all_times.append(parse_hhmm(b.start_time))
              all_times.append(parse_hhmm(b.end_time))
        candidate_start = parse_hhmm(tb.start_time)
        candidate_end = parse_hhmm(tb.end_time)
        all_times.extend([candidate_start, candidate_end])

        if all_times:
          min_time = min(all_times)
          max_time = max(all_times)
          delta = datetime.combine(datetime.today(), max_time) - datetime.combine(datetime.today(), min_time)
          if delta.total_seconds() > 36000:  # >10 hours
            continue

        blocks.append(tb)
    return blocks

  # Proximity scoring helper
  def proximity_score(tb):
    # Filter shifts assigned to this student on the same day
    assigned_today = [b for sc in STATE.schedules.values() for b in sc if b.id == stu.name and b.day == tb.day]
    if not assigned_today:
      return 50  # Neutral score when nothing is assigned

    # Find earliest start and latest end on the same day
    times = [(parse_hhmm(b.start_time), parse_hhmm(b.end_time)) for b in assigned_today]
    earliest = min(start for start, _ in times)
    latest = max(end for _, end in times)

    candidate_start = parse_hhmm(tb.start_time)
    candidate_end = parse_hhmm(tb.end_time)

    # Check if the shift falls within the preexisting range
    if earliest <= candidate_start and candidate_end <= latest:
      return 100  # Ideal: fully enclosed
    elif (earliest <= candidate_start <= latest) or (earliest <= candidate_end <= latest):
      return 80  # Partially overlapping
    elif candidate_end < earliest:
      # Distance in minutes to earliest
      delta = (datetime.combine(datetime.today(), earliest) - datetime.combine(datetime.today(), candidate_end)).total_seconds() / 60
    else:
      # Distance in minutes to latest
      delta = (datetime.combine(datetime.today(), candidate_start) - datetime.combine(datetime.today(), latest)).total_seconds() / 60

    # Shrink score the farther it gets from the assigned block span
    return max(1, 100 - delta)

  # Adjacency bonus
  def adjacency_bonus(tb):
    for sc in STATE.schedules.values():
      for b in sc:
        if b.id == stu.name and b.day == tb.day:
          if b.end_time == tb.start_time or b.start_time == tb.end_time:
            return 10
          bt = parse_hhmm(b.end_time)
          at = parse_hhmm(tb.start_time)
          if abs((datetime.combine(datetime.today(), at) - datetime.combine(datetime.today(), bt)).total_seconds()) <= 1800:
            return -10
    return 0

  # Availability pressure
  def availability_score(tb):
    count = 0
    for s in STATE.ordered_students:
      source = s.hd_availability if role_key == "hd" else s.availability
      for x in source:
        if x.day == tb.day and x.start_time == tb.start_time and x.end_time == tb.end_time and x.available in (1,2):
          count += 1
          break
    return 100 / count if count > 0 else 0

  def compute_block_score(tb, stu, role_key, w_avail=0.8, w_prox=0.3, w_exact=0.2, w_adj=0.2):
    availability = availability_score(tb)
    proximity = proximity_score(tb)
    availability_list = stu.hd_availability if role_key == 'hd' else stu.availability
    exact_match = next((x for x in availability_list if x.day == tb.day and x.start_time == tb.start_time and x.end_time == tb.end_time), None)
    exact_bonus = (1 if exact_match and exact_match.available == 1 else 0) * 50
    adjacency = adjacency_bonus(tb)
    return (
      w_avail * availability +
      w_prox * proximity +
      w_exact * exact_bonus +
      w_adj * adjacency
    )

  # Final ranking
  candidates = eligible_blocks()
  ranked = sorted(candidates, key=lambda tb: compute_block_score(tb, stu, role_key), reverse=True)

  if ranked:
    first = random.choice([r for r in ranked if
      abs(
          compute_block_score(r, stu, role_key) -
          compute_block_score(ranked[0], stu, role_key)
      ) < 1e-3
    ])
    first.id = stu.name
    if repeat == True:
        auto_assign_student(False)  # assign a second shift
  refresh_ui()

# Role selector
role_toggle = w.ToggleButtons(options=[("Help Desk","hd"),("Supply Runner","sr"),("Tech Desk","td")],value="hd")

# main containers
student_box = w.VBox()
grid_box    = w.VBox()
control_bar = w.HBox([prev_btn, next_btn, auto_btn])

def on_role_change(change):
    if change['name'] == 'value':
        refresh_ui()

def refresh_ui(*_):
  role = role_toggle.value
  flag = f"is_{role}"

  # Update the ordered list to reflect the currently selected role
  STATE.ordered_students = visible_students(STATE, role)
  STATE.current_index.setdefault(role, 0)

  # left list
  student_box.children = [build_student_list(role)]
  # grid
  grid_box.children = [build_grid(role)]
  # counter text
  stu = STATE.cur_stu
  if stu:
    count = STATE.round_assigned.get(stu.name,0)
    round_label.value = f"Assigned {count} shift(s) this round"
  else:
    round_label.value = "(no student selected)"

role_toggle.observe(on_role_change, names='value')

## Save and Export!

In [9]:
# Ensure the export folder exists
export_folder = "/content/final_schedules"
os.makedirs(export_folder, exist_ok=True)

def generate_time_slots(start="08:00", end="23:00", interval_minutes=30):
  time_format = "%H:%M"
  start_dt = datetime.strptime(start, time_format)
  end_dt = datetime.strptime(end, time_format)
  time_slots = []
  while start_dt < end_dt:
    slot_end = start_dt + timedelta(minutes=interval_minutes)
    time_slots.append(f"{start_dt.strftime(time_format)}-{slot_end.strftime(time_format)}")
    start_dt = slot_end
  return time_slots

def export_schedule_csvs(schedules, students, export_path):
  # main schedules
  for role, sched_list in schedules.items():
    rows = [{'Day': tb.day,
              'Start Time': tb.start_time,
              'End Time'  : tb.end_time,
              'Assigned To': tb.id or '[Open]'}
            for tb in sched_list]
    (pd.DataFrame(rows)
        .sort_values(['Day','Start Time'])
        .to_csv(os.path.join(export_path, f"{role.upper()}_Schedule.csv"),
                index=False))

  # student schedules
  time_slots = generate_time_slots("08:00", "23:00", 30)

  def to_time(x):                    # helper: str → datetime.time
      return x if isinstance(x, dtime) else datetime.strptime(x, "%H:%M").time()

  for stu in students:
      # build empty grid
      grid = {day: {slot: "" for slot in time_slots} for day in DAY_ORDER}

      # fill grid
      for role, sched_list in schedules.items():
          for tb in sched_list:
              if tb.id == stu.name:
                  start_t, end_t = to_time(tb.start_time), to_time(tb.end_time)
                  for slot in time_slots:
                      s0, s1 = map(to_time, slot.split("-"))
                      if s0 < end_t and s1 > start_t:
                          grid[tb.day][slot] = role.upper()

      df = pd.DataFrame({day: [grid[day][slot] for slot in time_slots]
                          for day in DAY_ORDER},
                        index=time_slots)
      df.index.name = "Time Slot"

      xlsx_path = os.path.join(export_path, f"{stu.name}_Schedule.xlsx")
      df.to_excel(xlsx_path)

      # autosize columns (optional)
      wb = load_workbook(xlsx_path)
      ws = wb.active
      for col in ws.columns:
          maxlen = max(len(str(c.value)) if c.value else 0 for c in col)
          ws.column_dimensions[col[0].column_letter].width = maxlen + 2
      wb.save(xlsx_path)

  return sorted(os.listdir(export_path))

# Run command to upload files

In [10]:
file_list = upload_excel_files()
ALL_STUDENTS = scrape_xlsx_files(file_list)

print("\nStudents successfully read:", len(ALL_STUDENTS))

Saving Ada.xlsx to Ada.xlsx
Saving Alan.xlsx to Alan.xlsx
Saving Andrew.xlsx to Andrew.xlsx
Saving Barbara.xlsx to Barbara.xlsx
Saving Caden.xlsx to Caden.xlsx
Saving Clara.xlsx to Clara.xlsx
Saving Connor.xlsx to Connor.xlsx
Saving Edsger.xlsx to Edsger.xlsx
Saving Elizabeth.xlsx to Elizabeth.xlsx
Saving Gottlob.xlsx to Gottlob.xlsx
Saving Grace.xlsx to Grace.xlsx
Saving Hayden.xlsx to Hayden.xlsx
Saving Hsu.xlsx to Hsu.xlsx
Saving Jacek.xlsx to Jacek.xlsx
Saving Kaylah.xlsx to Kaylah.xlsx
Saving Kristen.xlsx to Kristen.xlsx
Saving Leonard.xlsx to Leonard.xlsx
Saving Leslie.xlsx to Leslie.xlsx
Saving Lynn.xlsx to Lynn.xlsx
Saving Manuel.xlsx to Manuel.xlsx
Saving Margaret.xlsx to Margaret.xlsx
Saving Ruth.xlsx to Ruth.xlsx
Saving Sally.xlsx to Sally.xlsx
Saving Seymour.xlsx to Seymour.xlsx
Saving Susan.xlsx to Susan.xlsx

Students successfully read: 25


# Edit Student Roles and Information

In [11]:
refresh_warning()
refresh_student_table()

display(w.HBox([w.VBox([
  w.Label("Edit & Re‑order Students:"),
  btn_auto_sort,
  student_list_box
]), warn_out]))

# Visualize Schedules

In [12]:
STATE = initialise_scheduler(ALL_STUDENTS, TECH_DESK_STOPS, SUPPLY_RUN_STOPS)
refresh_ui()

ui = w.HBox([
  student_box,
  w.VBox([role_toggle, control_bar, grid_box],
          layout=w.Layout(border='1px solid gray', padding='4px', flex='1 1 auto'))
  ],
  layout=w.Layout(width='100%'))

display(ui)

# Export Schedules!

In [ ]:
export_schedule_csvs(
    schedules={
        "hd": STATE.schedules["hd"],
        "td": STATE.schedules["td"],
        "sr": STATE.schedules["sr"],
    },
    students=ALL_STUDENTS,
    export_path="/content/final_schedules"
)

zip_out = "/content/final_schedules.zip"
with zipfile.ZipFile(zip_out, "w") as zf:
    for fname in os.listdir(export_folder):
        path = os.path.join(export_folder, fname)
        if os.path.isfile(path):
            zf.write(path, arcname=fname)

files.download(zip_out)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>